# Step 1: Get h3 hex ids for parks in Beijing at an appropriate resolution

The goal of this demo is to create an index of park usage for individuals from the [Microsoft Geolife dataset](https://www.microsoft.com/en-us/download/details.aspx?id=52367), which contains high frequency GPS coordinates for 184 users in Beijing over five years. In this step, we estimate an appropriate h3 resolution given the size of the parks and get their h3 hex ids.

The park boundary files are available from [OpenStreetMap](https://download.geofabrik.de/asia/china/beijing.html). The two relevant shape files are located in `data/raw/shp`.

Note: If running the project notebooks in colab, CPU is fine. These data do not require much processing power, though some steps are time intensive

## 0. Set Up

### 0.1 Load packages

In [56]:
from google.colab import drive

In [57]:
import os
import geopandas as gpd

In [58]:
import shapely
from shapely.geometry import Polygon, MultiPolygon, Point, mapping
from typing import Iterable
import numpy as np
import pandas as pd

In [59]:
import folium

In [60]:
from tqdm.auto import tqdm

In [61]:
from pathlib import Path

In [62]:
!pip install h3
import h3

In [63]:
from h3.api import basic_str as h3s  # v4 string-based API


### 0.2 Mount Your Google Drive
You will be asked for permission to access your Google Drive.

Note: The `h3_index_demo` folder must be downloaded and unzipped to your personal Google Drive in order to run this code in Colab.

In [64]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 0.3 Change project directory
If the project folder is not in your main directory (`/content/drive`), change the directory here by updating `my_dir`.

In [65]:
try:
  os.chdir('/content/drive/MyDrive/h3_index_demo')
  print('Successfully changed project directory')
except:
  print('Project not in main Drive directory')
  try:
    # Define your containing folder here if not in main Drive directory
    my_dir = '/content/drive/MyDrive/!data_science'
    os.chdir(my_dir + '/h3_index_demo')
    print('Successfully changed project directory')
  except:
    print('Could not change to project directory.\nDid you define your containing folder?')

Project not in main Drive directory
Successfully changed project directory


## 1. Load shape files containing park polygons and filter
The shape files from OpenStreetMap that usually contain parks are located in `data/raw/shp`. Load these using geopandas.

In [66]:
# Load polygons (parks are under 'landuse' or 'leisure')
landuse = gpd.read_file("data/raw/shp/gis_osm_landuse_a_free_1.shp")
leisure = gpd.read_file("data/raw/shp/gis_osm_pois_a_free_1.shp")

In [67]:
# Filter for park polygons
parks = landuse[landuse['fclass'] == 'park']
# Some may be under leisure polygons
parks_extra = leisure[leisure['fclass'] == 'park']

In [68]:
print("Parks in landuse layer:", len(parks))
print("Parks in leisure layer:", len(parks_extra))

Parks in landuse layer: 3465
Parks in leisure layer: 3417


## 2. Check for and remove any duplicate parks
It is common for there to be a lot of duplicates between the landuse and leisure files because parks can appear in both.

### 2.1 Remove duplicate parks

In [69]:
# Deduplicate by finding parks that have the same centroid within 20 meters
# If this were a full project, we would test different thresholds (5 meters, 10 meters, etc.)

# Reproject to a projected CRS before geometric operations (China uses EPSG:3857 or EPSG:4490)
parks_proj = parks.to_crs(3857)
parks_extra_proj = parks_extra.to_crs(3857)

# Compute centroids safely
parks_proj = parks_proj.copy()
parks_extra_proj = parks_extra_proj.copy()
parks_proj["centroid"] = parks_proj.geometry.centroid
parks_extra_proj["centroid"] = parks_extra_proj.geometry.centroid

# Combine both layers
combined = gpd.GeoDataFrame(
    pd.concat([parks_proj, parks_extra_proj], ignore_index=True),
    crs=parks_proj.crs
)

# Round centroid coordinates for approximate duplicate removal
combined["x"] = (combined.centroid.x / 20).round(0)
combined["y"] = (combined.centroid.y / 20).round(0)

# Drop duplicates by centroid
deduplicated_parks = combined.drop_duplicates(subset=["x", "y"]).to_crs(4326)

print(f"{len(deduplicated_parks)} unique park polygons retained")

3463 unique park polygons retained


In [72]:
# Save a simplified copy for visualization later
gdf_save = deduplicated_parks.copy()

# Keep a single active geometry; drop any others (e.g., 'centroid')
geom_cols = gdf_save.columns[gdf_save.dtypes == "geometry"].tolist()
active_geom = gdf_save.geometry.name  # usually "geometry"
extra_geom = [c for c in geom_cols if c != active_geom]
gdf_save = gdf_save.drop(columns=extra_geom).set_geometry(active_geom)

# Save the simplified layer
gdf_save.to_file("data/etl/deduplicated_parks.gpkg", layer="parks", driver="GPKG")
gdf_save.to_file("data/etl/deduplicated_parks.geojson", driver="GeoJSON")

### 2.2 View deduplicated data and map example

In [14]:
deduplicated_parks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 3463 entries, 0 to 3464
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   osm_id    3463 non-null   object  
 1   code      3463 non-null   int32   
 2   fclass    3463 non-null   object  
 3   name      1288 non-null   object  
 4   geometry  3463 non-null   geometry
 5   centroid  3463 non-null   geometry
 6   x         3463 non-null   float64 
 7   y         3463 non-null   float64 
dtypes: float64(2), geometry(2), int32(1), object(3)
memory usage: 230.0+ KB


In [15]:
deduplicated_parks.head()

,osm_id,code,fclass,name,geometry,centroid,x,y
0,24476183,7202,park,建国门健身乐园,"POLYGON ((116.42728 39.90744, 116.42728 39.908...",POINT (12960700.17 4852591.234),648035.0,242630.0
1,24476204,7202,park,日坛公园,"POLYGON ((116.43498 39.91614, 116.43514 39.916...",POINT (12961748.421 4853487.828),648087.0,242674.0
2,24824550,7202,park,天坛公园,"POLYGON ((116.39415 39.88268, 116.39421 39.882...",POINT (12958080.881 4848584.527),647904.0,242429.0
3,24825400,7202,park,地坛公园,"POLYGON ((116.40616 39.95409, 116.40793 39.954...",POINT (12958591.584 4859047.432),647930.0,242952.0
4,24825443,7202,park,北京大观园,"POLYGON ((116.34807 39.87084, 116.34913 39.870...",POINT (12952023.144 4847071.41),647601.0,242354.0


In [16]:
# View the first row on the map in folium - is it a park?
# Select first park
park_row = deduplicated_parks.iloc[0]
park_center = [park_row.geometry.centroid.y, park_row.geometry.centroid.x]

# Create map
example_park_map = folium.Map(location=park_center, zoom_start=17)
folium.GeoJson(park_row.geometry).add_to(example_park_map)
example_park_map

In [17]:
# Yes, it looks like a park!
# If this were a full project, we'd spend more time verifying our map features, but this looks good for now

## 4. Exploration of h3 resolution needed for parks
This is important for our next step where we will check for whether our GPS coordinates are inside a particular h3 cell.


### 4.1 Look at the area of the parks in meters squared

In [18]:
# Add an area (meters squared column) to the gpd dataframe
def add_area_m2(parks_gdf: gpd.GeoDataFrame, area_crs: int = 6933) -> gpd.GeoDataFrame:
    """
    Adds 'park_area_m2' to the GeoDataFrame using an equal-area CRS (default EPSG:6933).
    Returns a copy in the original CRS.
    """
    if parks_gdf.crs is None:
        raise ValueError("parks_gdf must have a CRS set (e.g., EPSG:4326).")

    parks_eq = parks_gdf.to_crs(area_crs).copy()
    parks_eq["park_area_m2"] = parks_eq.geometry.area

    out = parks_gdf.copy()
    out["park_area_m2"] = parks_eq["park_area_m2"].values
    return out

# Summarize the area percentiles
def summarize_area_percentiles(parks_gdf: gpd.GeoDataFrame,
                               percentiles = (1,5,10,25,50,75,90,95,99)) -> pd.DataFrame:
    """
    Returns a table of requested percentiles for 'park_area_m2'.
    """
    a = parks_gdf["park_area_m2"].dropna().values
    p = np.percentile(a, percentiles)
    tbl = pd.DataFrame({
        "percentile": percentiles,
        "area_m2": p,
        "area_km2": p / 1_000_000
    })
    return tbl

In [19]:
# Add area to dataframe, use EPSG:4326
deduplicated_parks = add_area_m2(deduplicated_parks, area_crs=6933)

In [20]:
area_summary = summarize_area_percentiles(deduplicated_parks)
area_summary.head(10)

,percentile,area_m2,area_km2
0,1,3.422063e+02,0.000342
1,5,1.388643e+03,0.001389
2,10,2.624940e+03,0.002625
3,25,6.443596e+03,0.006444
4,50,1.749692e+04,0.017497
5,75,6.169991e+04,0.061700
6,90,2.059594e+05,0.205959
7,95,4.133365e+05,0.413336
8,99,1.855688e+06,1.855688


### 4.2 Estimate the number of h3 cells at different resolutions

In [21]:
# Get the average area of an H3 hex for a particular resolution in meters squared
def avg_hex_area_m2(res: int) -> float:
    try:
        return h3.hex_area(res, unit="m^2")      # h3>=4.x
    except AttributeError:
        # Fallback for older h3-py
        sample = h3.latlng_to_cell(0.0, 0.0, res)
        return h3.cell_area(sample, unit="m^2")

# Add estimated cell counts for a resolution of 10, 11, 12, and 13 to the summary table
for r in [10, 11, 12, 13]:
    a_hex = avg_hex_area_m2(r)
    area_summary[f"cells_r{r}"] = area_summary["area_m2"] / a_hex
    area_summary[f"cells_r{r}"] = area_summary[f"cells_r{r}"].round(1)

In [22]:
area_summary.head(10)

,percentile,area_m2,area_km2,cells_r10,cells_r11,cells_r12,cells_r13
0,1,3.422063e+02,0.000342,0.0,0.2,1.5,10.5
1,5,1.388643e+03,0.001389,0.1,0.9,6.1,42.5
2,10,2.624940e+03,0.002625,0.2,1.6,11.5,80.4
3,25,6.443596e+03,0.006444,0.6,4.0,28.2,197.4
4,50,1.749692e+04,0.017497,1.6,10.9,76.6,535.9
5,75,6.169991e+04,0.061700,5.5,38.6,270.0,1889.8
6,90,2.059594e+05,0.205959,18.4,128.7,901.2,6308.4
7,95,4.133365e+05,0.413336,36.9,258.4,1808.6,12660.2
8,99,1.855688e+06,1.855688,165.7,1160.0,8119.8,56838.4


A resolution of 12 looks like a good balance for the parks. If this were a larger project, we would test the speed and accuracy tradeoffs for lower and higher resolutions.

## 5. Create dataframe of hex ids for each park at resolution 12
The goal of this section is to get the hex ids for each park using the following steps:
- Convert each park polygon into its H3 r=12 hex cover ("polyfill").
- Compute overlap fraction per hex: area(park ∩ hex) / area(hex).
- Drop hexes with less than 5% overlap to reduce boundary slivers.
- Output:
  1) `park_hex`: long table of hex ids and overlap for each park
  2) `park_hex_unique`: unique list of park hexes for quick membership tests
  3) `hex_count_summary`: percentile table of hex counts per park after the 5% filter

Note: In a full project, different cutoffs would be tested for the overlap fraction, e.g. (1%, 10%, etc.)

## 5.1 Get hex ids for each park and save to csv file

In [23]:
RES = 12                  # Resolution of 12, estimated in previous step
OVERLAP_MIN = 0.05        # 5% overlap cutoff
AREA_CRS = 6933           # equal-area CRS for area math (accurate areas)
BASE_CRS = 4326           # lon/lat for H3 functions

In [24]:
# Ensure park geometries are in geodetic lon/lat (EPSG:4326) for H3.
# Make a copy so later edits (validity fixes, IDs) don’t mutate the original.
# Note: area math happens later in an equal-area CRS; this step is only for H3/polyfill.
parks_4326 = deduplicated_parks.to_crs(BASE_CRS).copy()  # BASE_CRS = 4326

In [25]:
parks_4326.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 3463 entries, 0 to 3464
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   osm_id        3463 non-null   object  
 1   code          3463 non-null   int32   
 2   fclass        3463 non-null   object  
 3   name          1288 non-null   object  
 4   geometry      3463 non-null   geometry
 5   centroid      3463 non-null   geometry
 6   x             3463 non-null   float64 
 7   y             3463 non-null   float64 
 8   park_area_m2  3463 non-null   float64 
dtypes: float64(3), geometry(2), int32(1), object(3)
memory usage: 257.0+ KB


In [26]:
# Use OSM feature id as the stable key
# Set as index (string) for consistent joins and summaries
parks_4326["osm_id"] = parks_4326["osm_id"].astype(str)
parks_4326 = parks_4326.set_index("osm_id")

In [27]:
# Geometry validity to avoid intersection errors
def _make_valid(geom):
    try:
        return shapely.make_valid(geom)
    except Exception:
        return geom.buffer(0)

parks_4326["geometry"] = parks_4326.geometry.apply(_make_valid)

In [28]:
# Equal-area copy for intersections and area calculations
parks_eq = parks_4326.to_crs(AREA_CRS).copy()

In [29]:
# Return set of H3 cells at res (12) covering a (multi)polygon in EPSG:4326.
# Handles holes. H3 expects coordinates in (lat, lon).
def _poly_to_cells_res(poly4326, res: int):
    if isinstance(poly4326, MultiPolygon):
        parts = list(poly4326.geoms)
    else:
        parts = [poly4326]

    cells = set()
    for g in parts:
        geo = mapping(g)                  # GeoJSON-like dict, lon/lat order
        cells |= set(h3s.geo_to_cells(geo, res))
    return cells

In [30]:
# Use the earlier function to get average area of a hex at given resolution
# A larger project could use the actual area of the hex
HEX_AREA_M2 = avg_hex_area_m2(RES)

In [37]:
OUT_DIR = Path("data/etl/parks_h3")

In [38]:
# Columns for each per-park CSV
# This took 1 hour 37 minutes
cols = ["osm_id", "hex_id", "res", "overlap_frac", "park_area_m2", "hex_area_m2", "overlap_area_m2"]

# Loop through park rows to get the h3 hexes and overlap
# Track progress through parks with tqdm
# Note: progress bar does not take into account park size
for osm_id, geom4326 in tqdm(parks_4326.geometry.items(),
                             total=len(parks_4326),
                             desc="Parks → hex CSVs (with overlap)",
                             unit="park"):
    out_path = OUT_DIR / f"{osm_id}.csv"
    if out_path.exists():
        continue  # already processed; supports resume

    # Equal-area geometry for this park
    geom_eq = parks_eq.loc[osm_id, "geometry"]
    park_area_m2 = float(geom_eq.area)

    # H3 polyfill at RES in EPSG:4326
    cell_ids = _poly_to_cells_res(geom4326, RES)

    rows_this_park = []
    if cell_ids:
        for cell in cell_ids:
            # Build hex polygon in lon/lat then project to equal-area
            latlon = h3s.cell_to_boundary(cell)  # [(lat, lon)]
            hex_lonlat = [(lon, lat) for (lat, lon) in latlon]
            hex_eq = gpd.GeoDataFrame({"geometry": [Polygon(hex_lonlat)]},
                                      crs=BASE_CRS).to_crs(AREA_CRS).iloc[0].geometry

            inter = geom_eq.intersection(hex_eq)
            if inter.is_empty:
                continue

            overlap_area_m2 = float(inter.area)
            hex_area_m2 = float(hex_eq.area)
            overlap_frac = overlap_area_m2 / hex_area_m2  # per-hex area denominator

            if overlap_frac >= OVERLAP_MIN:
                rows_this_park.append({
                    "osm_id": osm_id,
                    "hex_id": cell,
                    "res": RES,
                    "overlap_frac": overlap_frac,
                    "park_area_m2": park_area_m2,
                    "hex_area_m2": hex_area_m2,
                    "overlap_area_m2": overlap_area_m2,
                })

    # Write CSV for this park (even if empty, to mark done)
    pd.DataFrame(rows_this_park, columns=cols).to_csv(out_path, index=False)

Parks → hex CSVs (with overlap):   0%|          | 0/3463 [00:00<?, ?park/s]

### 5.2 Combine csv files across all parks

In [39]:
# Combine csv files
IN_DIR  = Path("data/etl/parks_h3")
OUT_ALL = Path("data/etl/park_hex_r12.csv")
OUT_UNQ = Path("data/etl/park_hex_unique_r12.csv")

cols = ["osm_id", "hex_id", "res", "overlap_frac", "park_area_m2", "hex_area_m2", "overlap_area_m2"]

# Gather per-park CSVs
paths = sorted(IN_DIR.glob("*.csv"))

if not paths:
    raise FileNotFoundError(f"No CSVs found in {IN_DIR.resolve()}")

# Read and combine; skip empty files cleanly
parts = []
for p in tqdm(paths, desc="Reading park CSVs", unit="file"):
    try:
        df = pd.read_csv(p, dtype={"osm_id": str, "hex_id": str, "res": "Int64",
                                   "overlap_frac": float, "park_area_m2": float,
                                   "hex_area_m2": float, "overlap_area_m2": float})
    except pd.errors.EmptyDataError:
        continue
    if df.empty:
        continue
    # Enforce expected columns/order; add any missing as NA
    for c in cols:
        if c not in df.columns:
            df[c] = pd.NA
    df = df[cols]
    parts.append(df)

if not parts:
    # No non-empty files
    combined = pd.DataFrame(columns=cols)
else:
    combined = pd.concat(parts, ignore_index=True)

# Drop exact duplicates (resume runs can duplicate)
combined = combined.drop_duplicates(subset=["osm_id", "hex_id", "res"])

# Save combined table
OUT_ALL.parent.mkdir(parents=True, exist_ok=True)
combined.to_csv(OUT_ALL, index=False)

Reading park CSVs:   0%|          | 0/3463 [00:00<?, ?file/s]

In [40]:
# Make unique-hex table
unique_hex = (
    combined.loc[:, ["hex_id", "res"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

unique_hex.to_csv(OUT_UNQ, index=False)

In [41]:
# Quick report
print("rows combined:", len(combined))
print("unique parks:", combined["osm_id"].nunique())
print("unique hexes:", len(unique_hex))
print("wrote:", OUT_ALL, "and", OUT_UNQ)

rows combined: 1823621
unique parks: 3448
unique hexes: 1816653
wrote: data/etl/park_hex_r12.csv and data/etl/park_hex_unique_r12.csv


In [42]:
combined.head()

,osm_id,hex_id,res,overlap_frac,park_area_m2,hex_area_m2,overlap_area_m2
0,1000058666,8c31aacddc607ff,12,1.0,149238.752552,219.753598,219.753598
1,1000058666,8c31aacddc611ff,12,1.0,149238.752552,219.751714,219.751714
2,1000058666,8c31aacddda15ff,12,1.0,149238.752552,219.785886,219.785886
3,1000058666,8c31aacddc2e5ff,12,1.0,149238.752552,219.766999,219.766999
4,1000058666,8c31aacddd8abff,12,1.0,149238.752552,219.777250,219.777250


In [44]:
# Rows where the same hex_id appears for multiple parks
# There are nearly 14k hex_ids that include multiple parks
multi_park_hex = (
    combined
    .groupby("hex_id")
    .filter(lambda g: g["osm_id"].nunique() > 1)
    .sort_values(["hex_id","osm_id"])
)

print(len(multi_park_hex))
multi_park_hex.head()

13936


,osm_id,hex_id,res,overlap_frac,park_area_m2,hex_area_m2,overlap_area_m2
648732,444429357,8c31aa0914d8bff,12,0.508648,1.463286e+06,218.439562,111.108855
1496284,899463539,8c31aa0914d8bff,12,0.605911,3.705214e+05,218.439562,132.354833
652895,444429357,8c31aa0914d9bff,12,0.522671,1.463286e+06,218.437164,114.170841
1497208,899463539,8c31aa0914d9bff,12,0.630882,3.705214e+05,218.437164,137.808150
652231,444429357,8c31aa0916199ff,12,0.550182,1.463286e+06,218.404557,120.162196


In [49]:
# Look at an example park that shares a hex with another park
osm = "444429357"
park_row = deduplicated_parks.loc[deduplicated_parks["osm_id"].astype(str) == osm].iloc[0]

pt = park_row.geometry.representative_point()  # safe center for display
park_center = [pt.y, pt.x]

hex_example_park_map = folium.Map(location=park_center, zoom_start=15)
folium.GeoJson(park_row.geometry, name=f"osm_id={osm}", tooltip=f"osm_id={osm}").add_to(hex_example_park_map)
hex_example_park_map

From the example above, we can see this is not a data error and it makes sense there are parks adjacent to other parks and therefore would share hex ids. A simple solution is to assign the hex id to the park that has more overlap. In a full analysis, we would want to test the sensitivity of the results to this.

### 5.3 Deduplicate hex ids
As shown in the previous step, adjacent parks can share hex ids. Keep the park that has the largest overlap with the hex id.

In [51]:
# Pick the row with the max overlap per hex_id
idx = combined.groupby("hex_id")["overlap_frac"].idxmax()
hex_unique_max = combined.loc[idx].reset_index(drop=True)
print(len(hex_unique_max))

1816653


In [54]:
# Check uniqueness of hex id
is_unique = hex_unique_max["hex_id"].is_unique
print(is_unique)

True


In [55]:
OUT_HEX_UNIQUE_MAX = Path("data/etl/park_hex_unique_r12.csv")
hex_unique_max.to_csv(OUT_HEX_UNIQUE_MAX, index=False)